In [113]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
%matplotlib inline

from ray import tune
import ray

In [114]:
ray.shutdown()
ray.init()
filename = "./data/wine.csv"
df = pd.read_csv(filename)
df = df.drop('index', axis=1)
print(df.shape)
print(df["quality"].value_counts())

2021-03-03 15:48:32,682	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(6497, 12)
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64


In [115]:
from sklearn.utils import resample

#Resample:
df_majority = df[df['quality'] == 6]
for i in range(3,10):
    majority_len = df[df['quality'] == 6].shape[0]
    if i != 6:
        minority_len = df[df['quality'] == i].shape[0]

        df_minority = df[df['quality'] == i]

        df_minority_upsampled = resample(df_minority, 
                                        replace=True,               
                                        n_samples=majority_len,
                                        random_state=123)

        df_majority = df_majority.append(df_minority_upsampled)
df = df_majority
print(df.shape)
print(df["quality"].value_counts())
df = df.sample(frac=1).reset_index(drop=True)

(19852, 12)
3    2836
4    2836
5    2836
6    2836
7    2836
8    2836
9    2836
Name: quality, dtype: int64


In [116]:
train_test_split_fraction = 0.80
split_index = int(df.shape[0] * train_test_split_fraction)
df_train = df[:split_index]
df_test = df[split_index:]

target = pd.get_dummies(df['quality']).values

X_train = df_train.drop('quality', axis = 1).values
X_test = df_test.drop('quality', axis = 1).values

y_train = target[:split_index]
y_test = target[split_index:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15881, 11)
(3971, 11)
(15881, 7)
(3971, 7)


In [117]:
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

In [118]:
#nb_hidden_neurons = 200
nb_classes = len(pd.unique(df['quality']))
nb_classes

7

In [119]:
import torch.nn as nn

class Network(nn.Module):

    def __init__(self, nb_features, nb_hidden_neurons):
        """Here we define the layers
        """

        super().__init__()
        
        self.layer_1 =  nn.Linear(nb_features, nb_hidden_neurons)
        self.layer_2 =  nn.Linear(nb_hidden_neurons, nb_hidden_neurons)
        self.layer_out =  nn.Linear(nb_hidden_neurons, nb_classes)
        
        

    def forward(self,x, nb_hidden_layers, activation_function):
        """Here we combine the layers
        """
        
        activation_function = activation_function 
        last_layer_activation = nn.Softmax()
        
        output_first_layer = activation_function(self.layer_1(x))
        outut_hidden_layer = activation_function(self.layer_2(output_first_layer))
        for i in range(1, nb_hidden_layers):
            outut_hidden_layer = activation_function(self.layer_2(outut_hidden_layer))
        prediction = last_layer_activation(self.layer_out(outut_hidden_layer))
        return prediction

In [120]:
def test(X, y, config):
    my_nn = Network(nb_features=X_train.shape[1], nb_hidden_neurons=config['hidden_neuron'])
    correct = 0
    total = 0
    batch_size = config['batch_size']
    with torch.no_grad():
        for idx in range(0, X.shape[0], batch_size):
            
            total += 1
            
            output = my_nn(X.float(), nb_hidden_layers= config['nb_layers'],activation_function= config['activation'])
            
            predicted = torch.max(output, 1)
            
            target_pred = torch.max(y, 1)
            
            if predicted[1][idx] == target_pred[1][idx]:
                correct += 1
            
    accuracy = correct/total       
    return accuracy

In [129]:
def training(config):
    
    nb_steps_loss_sum = 10
    my_nn = Network(nb_features=X_train.shape[1], nb_hidden_neurons=config['hidden_neuron'])
    loss_list = []
    running_loss = 0
    batch_nb = 0
    criterion = nn.MSELoss()
    batch_size =  config['batch_size']
    learning_rate = config['lr'] 
    optimizer = torch.optim.AdamW(my_nn.parameters(), lr=learning_rate)

    for epoch in range(0,config['epoch']):
        for idx in range(0, X_train.shape[0], batch_size):
            
            input_batch = X_train[idx:idx + batch_size]
            target = y_train[idx:idx + batch_size]

            optimizer.zero_grad()

            y_pred = my_nn(input_batch.float(),nb_hidden_layers= config['nb_layers'],activation_function= config['activation'])

            loss = criterion(y_pred, target.float())

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            save_loss_condition = batch_nb % nb_steps_loss_sum == (nb_steps_loss_sum - 1)
            
            if save_loss_condition:    
                loss_list.append(running_loss)
                running_loss = 0.0

            batch_nb+= 1
    print(loss_list)
    #acc_train = test(X_train, y_train, config)
    #acc_test = test(X_test, y_test, config)
    #tune.report(acc_train=acc_train, acc_test=acc_test)

In [130]:
search_space = {
    "lr": tune.grid_search([0.001, 0.01, 0.1]),
    "hidden_neuron": tune.grid_search([50, 100]),
    "epoch": tune.grid_search([10, 20]),
    "batch_size": tune.grid_search([20, 30]),
    "nb_layers": tune.grid_search([3, 5]),
    "activation": tune.grid_search([nn.ReLU()])
}
analysis = tune.run(training, config=search_space)

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00000,RUNNING,,ReLU(),20,10,50,0.001,3


0802383, 1.1107563003897667, 1.0134999677538872, 1.0977595672011375, 1.0917908772826195, 1.0855652764439583, 1.0943051800131798, 1.0918880105018616, 1.1066794469952583, 1.043928436934948, 1.1013054326176643, 1.045135736465454, 1.093077003955841, 1.0939839780330658, 1.0839145556092262, 1.0240365043282509, 1.0351536944508553, 1.0343263819813728, 1.0276907160878181, 1.041858084499836, 1.0366736724972725, 1.0170724540948868, 1.0874927416443825, 1.0870045349001884, 1.0493703857064247, 1.0039593651890755, 1.0545802041888237, 1.02824667096138, 1.0291259735822678, 1.0407246798276901, 1.0747710019350052, 1.051348716020584, 1.0297188386321068, 0.9922707676887512, 1.0354414135217667, 0.9575286954641342, 1.015459269285202, 1.0230566412210464, 1.0455942824482918, 1.0614333674311638, 1.0018548741936684, 1.022209495306015, 1.0654179230332375, 1.0030944496393204, 1.0334749072790146, 0.9514681175351143, 0.9890758544206619, 0.9954083859920502, 0.9578296393156052, 0.9724991098046303, 1.0100507736206055, 

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00000,RUNNING,,ReLU(),20,10,50,0.001,3
training_e5557_00002,RUNNING,,ReLU(),20,20,50,0.001,3
training_e5557_00003,RUNNING,,ReLU(),30,20,50,0.001,3
training_e5557_00004,RUNNING,,ReLU(),20,10,100,0.001,3
training_e5557_00005,RUNNING,,ReLU(),30,10,100,0.001,3
training_e5557_00006,RUNNING,,ReLU(),20,20,100,0.001,3
training_e5557_00007,RUNNING,,ReLU(),30,20,100,0.001,3
training_e5557_00008,PENDING,,ReLU(),20,10,50,0.01,3
training_e5557_00001,TERMINATED,,ReLU(),30,10,50,0.001,3


870207787, 1.0211028531193733, 1.014164924621582, 1.024440050125122, 1.0295911207795143, 1.0075326785445213, 1.0500543862581253, 1.0107817575335503, 1.0717875212430954, 0.9961630925536156, 1.044700987637043, 1.0187339931726456, 1.0312124192714691, 0.9449469596147537, 0.9698008745908737, 1.0046795010566711, 0.9722125977277756, 0.995910182595253, 0.9738537520170212, 0.9621260538697243, 1.005066603422165, 1.011624090373516, 1.0236235558986664, 0.9820526167750359, 0.9727327600121498, 0.9861672148108482, 1.0229920893907547, 1.006182387471199, 1.035881668329239, 1.0359815284609795, 1.03145033121109, 0.9540674388408661, 0.99990014731884, 0.9159812927246094, 0.9931050688028336, 0.9802174642682076, 0.9972680062055588, 1.0200951471924782, 0.9508609399199486, 0.9907172173261642, 1.0214143842458725, 0.9552597627043724, 1.0030440166592598, 0.8968299776315689, 0.944005161523819, 0.9648137837648392, 0.9510689675807953, 0.9260104596614838, 0.9512634128332138, 0.9876754432916641, 0.991267129778862, 0.9

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00002,RUNNING,,ReLU(),20,20,50,0.001,3
training_e5557_00003,RUNNING,,ReLU(),30,20,50,0.001,3
training_e5557_00004,RUNNING,,ReLU(),20,10,100,0.001,3
training_e5557_00006,RUNNING,,ReLU(),20,20,100,0.001,3
training_e5557_00007,RUNNING,,ReLU(),30,20,100,0.001,3
training_e5557_00008,RUNNING,,ReLU(),20,10,50,0.01,3
training_e5557_00009,RUNNING,,ReLU(),30,10,50,0.01,3
training_e5557_00010,PENDING,,ReLU(),20,20,50,0.01,3
training_e5557_00000,TERMINATED,,ReLU(),20,10,50,0.001,3
training_e5557_00001,TERMINATED,,ReLU(),30,10,50,0.001,3


...

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00046,RUNNING,,ReLU(),20,20,100,0.1,5
training_e5557_00047,RUNNING,,ReLU(),30,20,100,0.1,5
training_e5557_00000,TERMINATED,,ReLU(),20,10,50,0.001,3
training_e5557_00001,TERMINATED,,ReLU(),30,10,50,0.001,3
training_e5557_00002,TERMINATED,,ReLU(),20,20,50,0.001,3
training_e5557_00003,TERMINATED,,ReLU(),30,20,50,0.001,3
training_e5557_00004,TERMINATED,,ReLU(),20,10,100,0.001,3
training_e5557_00005,TERMINATED,,ReLU(),30,10,100,0.001,3
training_e5557_00006,TERMINATED,,ReLU(),20,20,100,0.001,3
training_e5557_00007,TERMINATED,,ReLU(),30,20,100,0.001,3


5, 2.4666667133569717, 2.4285714626312256, 2.2761905044317245, 2.447619080543518, 2.438095271587372, 2.3714286237955093, 2.4666666984558105, 2.4952381402254105, 2.352381005883217, 2.3809524178504944, 2.4857143461704254, 2.4666667133569717, 2.4761905521154404, 2.4761905521154404, 2.4476190954446793, 2.49783556163311, 2.390476241707802, 2.4476191103458405, 2.4190476685762405, 2.514285773038864, 2.4190476536750793, 2.4952381402254105, 2.4952381402254105, 2.5142858028411865, 2.4857143610715866, 2.4476191252470016, 2.438095271587372, 2.438095286488533, 2.371428593993187, 2.428571477532387, 2.4666667133569717, 2.4666667133569717, 2.4571429193019867, 2.4857143461704254, 2.438095301389694, 2.5238095968961716, 2.466666743159294, 2.3809524178504944, 2.438095301389694, 2.361904814839363, 2.390476256608963, 2.495238170027733, 2.4952381551265717, 2.438095286488533, 2.4190476834774017, 2.4285714626312256, 2.5333333909511566, 2.438095286488533, 2.4190476536750793, 2.4666667133569717, 2.41904766857624

Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00000,TERMINATED,,ReLU(),20,10,50,0.001,3
training_e5557_00001,TERMINATED,,ReLU(),30,10,50,0.001,3
training_e5557_00002,TERMINATED,,ReLU(),20,20,50,0.001,3
training_e5557_00003,TERMINATED,,ReLU(),30,20,50,0.001,3
training_e5557_00004,TERMINATED,,ReLU(),20,10,100,0.001,3
training_e5557_00005,TERMINATED,,ReLU(),30,10,100,0.001,3
training_e5557_00006,TERMINATED,,ReLU(),20,20,100,0.001,3
training_e5557_00007,TERMINATED,,ReLU(),30,20,100,0.001,3
training_e5557_00008,TERMINATED,,ReLU(),20,10,50,0.01,3
training_e5557_00009,TERMINATED,,ReLU(),30,10,50,0.01,3


Trial name,status,loc,activation,batch_size,epoch,hidden_neuron,lr,nb_layers
training_e5557_00000,TERMINATED,,ReLU(),20,10,50,0.001,3
training_e5557_00001,TERMINATED,,ReLU(),30,10,50,0.001,3
training_e5557_00002,TERMINATED,,ReLU(),20,20,50,0.001,3
training_e5557_00003,TERMINATED,,ReLU(),30,20,50,0.001,3
training_e5557_00004,TERMINATED,,ReLU(),20,10,100,0.001,3
training_e5557_00005,TERMINATED,,ReLU(),30,10,100,0.001,3
training_e5557_00006,TERMINATED,,ReLU(),20,20,100,0.001,3
training_e5557_00007,TERMINATED,,ReLU(),30,20,100,0.001,3
training_e5557_00008,TERMINATED,,ReLU(),20,10,50,0.01,3
training_e5557_00009,TERMINATED,,ReLU(),30,10,50,0.01,3


48, 2.357142895460129, 2.400000050663948, 2.4142857640981674, 2.442857190966606, 2.4857143461704254, 2.514285773038864, 2.428571507334709, 2.4428572207689285, 2.5285715013742447, 2.41428579390049, 2.4428572058677673, 2.5428572446107864, 2.5142857879400253, 2.428571477532387, 2.4142857789993286, 2.38571435213089, 2.428571492433548, 2.3428571969270706, 2.38571435213089, 2.400000050663948, 2.5142858028411865, 2.5142857879400253, 2.457142934203148, 2.4428572058677673, 2.457142934203148, 2.3857143372297287, 2.457142934203148, 2.4857143610715866, 2.500000074505806, 2.4000000655651093, 2.4714286476373672, 2.4142857640981674, 2.5000001043081284, 2.4428572058677673, 2.3857143372297287, 2.5142857879400253, 2.3714286237955093, 2.4857143610715866, 2.4857143461704254, 2.4142857789993286, 2.4428572058677673, 2.271428629755974, 2.328571453690529, 2.4857143461704254, 2.3857143372297287, 2.4142857789993286, 2.414285808801651, 2.500000074505806, 2.471428632736206, 2.471428632736206, 2.3000000417232513, 

In [128]:
print(analysis.get_best_config(metric="mean_loss", mode="min"))

2021-03-03 16:00:56,468	WARNING experiment_analysis.py:576 -- Could not find best trial. Did you pass the correct `metric` parameter?
None
